In [6]:
import numpy as np
import yfinance as yf

# Fetch current stock price of SPY
spy = yf.Ticker('SPY')
data = spy.history(period='1d')
S0 = data['Close'].iloc[-1]
print(f"Current stock price of SPY (S0): {S0:.2f}")

# Fetch risk-free rate (3-month Treasury bill rate)
risk_free_rate = yf.Ticker('^IRX')
risk_free_rate_data = risk_free_rate.history(period='1d')
current_rf_rate = risk_free_rate_data['Close'].iloc[-1] / 100
print(f"Current Risk-Free Rate (3-month Treasury bill): {current_rf_rate:.4f}")

# Historical data for SPY to calculate drift and volatility
hist = spy.history(period='1y')
hist['log_returns'] = np.log(hist['Close'] / hist['Close'].shift(1))

alpha = hist['log_returns'].mean() * 252
sigma = hist['log_returns'].std() * np.sqrt(252)
print(f"Annualized Drift (alpha): {alpha:.6f}")
print(f"Annualized Volatility (sigma): {sigma:.6f}")

def binomial_tree_option_pricing(S0, K, T, r, sigma, N, option_type='call'):
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)

    # Initialize asset prices at maturity
    ST = np.zeros(N + 1)
    ST[0] = S0 * (d ** N)
    for j in range(1, N + 1):
        ST[j] = ST[j - 1] * (u / d)

    # Initialize option values at maturity
    option_values = np.maximum(0, (ST - K) if option_type == 'call' else (K - ST))

    # Step back through the tree
    for i in range(N - 1, -1, -1):
        for j in range(i + 1):
            option_values[j] = (p * option_values[j + 1] + (1 - p) * option_values[j]) * np.exp(-r * dt)

    return option_values[0]

# Example usage with updated parameters
K = 550
T = 7 / 252
N = 50

option_price_binomial = binomial_tree_option_pricing(S0, K, T, current_rf_rate, sigma, N, option_type='call')
print(f"Binomial Model Call Option Price: {option_price_binomial:.2f}")


Current stock price of SPY (S0): 538.41
Current Risk-Free Rate (3-month Treasury bill): 0.0516
Annualized Drift (alpha): 0.181815
Annualized Volatility (sigma): 0.115399
Binomial Model Call Option Price: 0.81


In [7]:
def trinomial_tree_option_pricing(S0, K, T, r, sigma, N, option_type='call'):
    dt = T / N
    u = np.exp(sigma * np.sqrt(2 * dt))
    d = 1 / u
    pu = ((np.exp(r * dt / 2) - np.exp(-sigma * np.sqrt(dt / 2))) / 
          (np.exp(sigma * np.sqrt(dt / 2)) - np.exp(-sigma * np.sqrt(dt / 2)))) ** 2
    pd = ((np.exp(sigma * np.sqrt(dt / 2)) - np.exp(r * dt / 2)) / 
          (np.exp(sigma * np.sqrt(dt / 2)) - np.exp(-sigma * np.sqrt(dt / 2)))) ** 2
    pm = 1 - pu - pd

    # Initialize asset prices at maturity
    ST = np.zeros(2 * N + 1)
    ST[0] = S0 * (d ** N)
    for j in range(1, 2 * N + 1):
        ST[j] = ST[j - 1] * (u / d)

    # Initialize option values at maturity
    option_values = np.maximum(0, (ST - K) if option_type == 'call' else (K - ST))

    # Step back through the tree
    for i in range(N - 1, -1, -1):
        for j in range(2 * i + 1):
            option_values[j] = (pu * option_values[j + 2] + pm * option_values[j + 1] + pd * option_values[j]) * np.exp(-r * dt)

    return option_values[0]

# Example usage with updated parameters
option_price_trinomial = trinomial_tree_option_pricing(S0, K, T, current_rf_rate, sigma, N, option_type='call')
print(f"Trinomial Model Call Option Price: {option_price_trinomial:.2f}")


Trinomial Model Call Option Price: 104.56
